# Start spark application (spark session)

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1649133354814_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
type(spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.session.SparkSession'>

# Load ticket data from the S3 bucket

In [3]:
s3_path = "s3a://upgrad-data/Parking_Violation_Tickets.csv"
tickets = spark.read.csv(s3_path, header = True, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check schema of the DataFrame

In [4]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

# Import all SQL functions and create SQL view

In [5]:
from pyspark.sql.functions import *
tickets.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Check for duplicate entries

In [6]:
spark.sql("\
            SELECT `Summons Number`, COUNT(`Summons Number`) AS `Number of duplicate occurance of summons number`\
            FROM tickets_view\
            GROUP BY `Summons Number`\
            HAVING COUNT(`Summons Number`) > 1\
        ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------------------------------------+
|Summons Number|Number of duplicate occurance of summons number|
+--------------+-----------------------------------------------+
+--------------+-----------------------------------------------+

# Total number of tickets

In [7]:
tickets.select(tickets["Summons Number"]).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10803028

# Number of unique states from where cars got parking ticket

In [8]:
spark.sql("\
            SELECT COUNT(`Registration State`) as `Number of unique states`\
            FROM\
            (\
                SELECT `Registration State`\
                FROM tickets_view\
                GROUP BY `Registration State`\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|Number of unique states|
+-----------------------+
|                     67|
+-----------------------+

# State having the maximum tickets

In [9]:
spark.sql("\
            SELECT `Registration State`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Registration State`\
            ORDER BY `Number of Tickets` DESC\
            LIMIT 1\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Number of tickets|
+------------------+-----------------+
|                NY|          8481061|
+------------------+-----------------+

# Replace the numeric entry '99' with the state having maximum tickets

In [10]:
tickets_updated = tickets.withColumn("Registration State", when(tickets["Registration State"] == '99', 'NY').otherwise(tickets["Registration State"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create SQL view of the updated DataFrame

In [11]:
tickets_updated.createOrReplaceTempView("tickets_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Number of unique states from where cars got parking ticket


In [12]:
spark.sql("\
            SELECT COUNT(`Registration State`) as `Number of unique states`\
            FROM\
            (\
                SELECT `Registration State`\
                FROM tickets_view\
                GROUP BY `Registration State`\
            ) AS SUBQUERY\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|Number of unique states|
+-----------------------+
|                     66|
+-----------------------+

# Top twenty states with the most number of tickets

In [13]:
spark.sql("\
            SELECT `Registration State`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Registration State`\
            ORDER BY `Number of Tickets` DESC\
            LIMIT 20\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+
|Registration State|Number of tickets|
+------------------+-----------------+
|                NY|          8517686|
|                NJ|           925965|
|                PA|           285419|
|                FL|           144556|
|                CT|           141088|
|                MA|            85547|
|                IN|            80749|
|                VA|            72626|
|                MD|            61800|
|                NC|            55806|
|                IL|            37329|
|                GA|            36852|
|                TX|            36516|
|                AZ|            26426|
|                OH|            25302|
|                CA|            24260|
|                SC|            21836|
|                ME|            21574|
|                MN|            18227|
|                OK|            18165|
+------------------+-----------------+

# Frequency of top five violation codes

In [14]:
spark.sql("\
            SELECT `Violation Code`, COUNT(`Summons Number`) as `Number of violations`\
            FROM tickets_view\
            GROUP BY `Violation Code`\
            ORDER BY `Number of violations` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+
|Violation Code|Number of violations|
+--------------+--------------------+
|            21|             1528588|
|            36|             1400614|
|            38|             1062304|
|            14|              893498|
|            20|              618593|
+--------------+--------------------+

# Frequency of top five vehicle body type

In [15]:
spark.sql("\
            SELECT `Vehicle Body Type`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Vehicle Body Type`\
            ORDER BY `Number of tickets` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+
|Vehicle Body Type|Number of tickets|
+-----------------+-----------------+
|             SUBN|          3719802|
|             4DSD|          3082020|
|              VAN|          1411970|
|             DELV|           687330|
|              SDN|           438191|
+-----------------+-----------------+

# Frequency of top five vehicle make

In [16]:
spark.sql("\
            SELECT `Vehicle Make`, COUNT(`Summons Number`) as `Number of tickets`\
            FROM tickets_view\
            GROUP BY `Vehicle Make`\
            ORDER BY `Number of tickets` DESC\
            LIMIT 5\
            ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+
|Vehicle Make|Number of tickets|
+------------+-----------------+
|        FORD|          1280958|
|       TOYOT|          1211451|
|       HONDA|          1079238|
|       NISSA|           918590|
|       CHEVR|           714655|
+------------+-----------------+

# Divide the data into seasons

# Assumptions

|Season|Months|
|------|------|
|Summer|March, April, May|
|Monsoon|June, July, August|
|Autumn|September, October, November|
|Winter|December, January, February|

# Casting timestamp as datatype